In [6]:
import pandas as pd
import os
from datetime import datetime
import re

In [7]:
def process_csv_files(folder_path, frequency='Daily', rank_filter=1):
    try:
        # Step 1: Read all CSV files in the folder
        all_data = []
        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(folder_path, filename)
                df = pd.read_csv(file_path)
                df['Strategy'] = filename.replace('.csv', '')
                all_data.append(df)

        if not all_data:
            print(f"No CSV files found in {folder_path}")
            return pd.DataFrame(), pd.DataFrame()

        # Combine all dataframes
        combined_df = pd.concat(all_data, ignore_index=True)

        # Step 2: Group by entry date and sum profit
        combined_df.columns = combined_df.columns.str.lower()

        # Convert entry_date to datetime, handling both "YYYY-MM-DD" and "DD/MM/YY" formats
        combined_df['entry_date'] = pd.to_datetime(combined_df['entry_date'], format='%Y-%m-%d', errors='coerce')
        mask = combined_df['entry_date'].isna()
        combined_df.loc[mask, 'entry_date'] = pd.to_datetime(combined_df.loc[mask, 'entry_date'], format='%d/%m/%y')

        # Ensure entry_date is just the date, no time component
        combined_df['entry_date'] = combined_df['entry_date'].dt.date

        grouped_df = combined_df.groupby(['entry_date', 'strategy'])['profit'].sum().reset_index()

        # Step 3: Pivot the dataframe
        pivot_df = grouped_df.pivot(index='entry_date', columns='strategy', values='profit').reset_index()
        pivot_df = pivot_df.fillna(0)

        # Apply frequency
        if frequency != 'Daily':
            pivot_df['entry_date'] = pd.to_datetime(pivot_df['entry_date'])
            if frequency == 'Weekly':
                pivot_df['entry_date'] = pivot_df['entry_date'] - pd.to_timedelta(pivot_df['entry_date'].dt.dayofweek,
                                                                                  unit='D')
            elif frequency == 'Monthly':
                pivot_df['entry_date'] = pivot_df['entry_date'].dt.to_period('M').dt.to_timestamp()
            elif frequency == 'Quarterly':
                pivot_df['entry_date'] = pivot_df['entry_date'].dt.to_period('Q').dt.to_timestamp()

            pivot_df = pivot_df.groupby('entry_date').sum().reset_index()

        # Ensure entry_date is just the date again after frequency adjustments
        pivot_df['entry_date'] = pd.to_datetime(pivot_df['entry_date']).dt.date

        # Step 4: Rank strategies based on profit from the previous row
        strategy_columns = pivot_df.columns.drop('entry_date')
        pivot_df = pivot_df.sort_values('entry_date')

        for i in range(len(pivot_df)):
            if i == 0:
                # For the first row, rank based on its own profits
                ranks = pivot_df.iloc[i][strategy_columns].rank(ascending=False, method='min')
            else:
                # For subsequent rows, rank based on the previous row's profits
                ranks = pivot_df.iloc[i - 1][strategy_columns].rank(ascending=False, method='min')

            for strategy in strategy_columns:
                pivot_df.loc[pivot_df.index[i], f'{strategy}_rank'] = ranks[strategy]

        # Step 5: Prepare final output with strategy selection based on previous day's ranking
        pivot_df['Date'] = pivot_df['entry_date']

        selected_strategies = []
        for i in range(len(pivot_df)):
            if i == 0:
                # For the first day, use its own ranking
                selected_strategy = pivot_df.iloc[i][strategy_columns].idxmax()
            else:
                # For subsequent days, use the previous day's ranking
                prev_ranks = pivot_df.iloc[i][
                    [f'{col}_rank' for col in strategy_columns if f'{col}_rank' in pivot_df.columns]]

                if not prev_ranks.empty:
                    # Find the strategy with the rank closest to rank_filter
                    rank_differences = abs(prev_ranks - rank_filter)
                    closest_rank_strategy = rank_differences.idxmin()
                    selected_strategy = closest_rank_strategy.replace('_rank', '')
                else:
                    # If no ranks are available, use the strategy with the highest profit
                    selected_strategy = pivot_df.iloc[i][strategy_columns].idxmax()

            selected_strategies.append(selected_strategy)

        pivot_df['Selected_Strategy'] = selected_strategies
        pivot_df['Selected_Profit'] = pivot_df.apply(lambda row: row[row['Selected_Strategy']], axis=1)

        final_df = pivot_df[['Date', 'Selected_Strategy', 'Selected_Profit']]
        final_df.columns = ['Date', 'Strategy', 'Profit']

        # Remove duplicates
        final_df = final_df.drop_duplicates()

        # Ensure Date is in the correct format
        final_df['Date'] = pd.to_datetime(final_df['Date']).dt.strftime('%Y-%m-%d')

        return final_df, pivot_df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()


In [8]:
def save_to_excel(df, file_name):
    # Create 'output' folder if it doesn't exist
    if not os.path.exists('output'):
        os.makedirs('output')

    # Save to Excel
    full_path = os.path.join('output', file_name)
    df.to_excel(full_path, index=False)
    print(f"Data saved to {full_path}")

In [9]:
def create_best_model_report(df):
    # Melt the dataframe to long format
    melted_df = df.melt(id_vars=['Date'], var_name='Strategy', value_name='Profit')
    
    # Sort by Date and Profit (descending) and keep only the best model for each date
    best_model_df = melted_df.sort_values(['Date', 'Profit'], ascending=[True, False])
    best_model_df = best_model_df.groupby('Date').first().reset_index()
    
    # Sort by Date
    best_model_df = best_model_df.sort_values('Date')
    
    return best_model_df

In [10]:
# Usage
folder = 'data/STRATEGIES/'
# result = process_csv_files(folder)
# print(result)

frequencies = ['Daily', 'Weekly', 'Monthly', 'Quarterly']
rank_filter = 1  # Change this to the desired rank


for frequency in frequencies:
    final_df, full_df = process_csv_files(folder, frequency=frequency, rank_filter=rank_filter)

    # Save the main report
    save_to_excel(final_df, f'best_strategy_{frequency.lower()}.xlsx')

    # Save the full report
    save_to_excel(full_df, f'full_analysis_{frequency.lower()}.xlsx')

Data saved to output/best_strategy_daily.xlsx
Data saved to output/full_analysis_daily.xlsx
Data saved to output/best_strategy_weekly.xlsx
Data saved to output/full_analysis_weekly.xlsx
Data saved to output/best_strategy_monthly.xlsx
Data saved to output/full_analysis_monthly.xlsx
Data saved to output/best_strategy_quarterly.xlsx
Data saved to output/full_analysis_quarterly.xlsx
